In [1]:
from IPython.display import display
import gc

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_frame = pd.read_pickle('data/train_frame.pkl')
train_frame = train_frame.sample(frac=1)
train_frame.head()

,label,pixels
0,dog,"[[38.0, 50.0, 60.0], [37.0, 50.0, 63.0], [34.0..."
0,dog,"[[72.0, 52.0, 43.0], [76.0, 58.0, 49.0], [55.0..."
0,dog,"[[252.0, 243.0, 208.0], [254.0, 243.0, 211.0],..."
0,dog,"[[21.0, 21.0, 19.0], [22.0, 22.0, 20.0], [35.0..."
0,dog,"[[172.0, 203.0, 107.0], [133.0, 172.0, 76.0], ..."


In [4]:
train_pixels = np.array([pixel for pixel in train_frame.pixels.values]).reshape(-1,128,128,3) / 255.0
train_label = np.array([label for label in train_frame.label.values])
train_pixels.shape, train_label.shape

((25000, 128, 128, 3), (25000,))

In [5]:
del train_frame
gc.collect()

0

In [6]:
from keras.utils import to_categorical

In [7]:
train_label[train_label == 'cat'] = 0
train_label[train_label == 'dog'] = 1

train_label = train_label.astype('float64')
#train_label = to_categorical(train_label)
train_label.shape

(25000,)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_train, x_valid, y_train, y_valid = train_test_split(train_pixels, train_label, test_size=0.2, random_state=42)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((20000, 128, 128, 3), (5000, 128, 128, 3), (20000,), (5000,))

In [10]:
del train_pixels, train_label
gc.collect()

0

In [11]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.activations import relu, sigmoid

In [25]:
model = Sequential()

model.add(Input(shape=(128,128,3)))
model.add(Conv2D(filters = 32, kernel_size = (3,3), activation=relu))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation=relu))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(filters = 128, kernel_size = (3,3), activation=relu))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(filters = 256, kernel_size = (3,3), activation=relu))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(rate=0.2))
model.add(Dense(256, activation=relu))
model.add(Dense(1, activation=sigmoid))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 63, 63, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 30, 30, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 14, 14, 128)     

In [13]:
from keras.optimizers import Adam

In [14]:
opt = Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
)

In [15]:
from keras.callbacks import EarlyStopping

In [16]:
callback = EarlyStopping(
    monitor="accuracy",
    min_delta=0.0001,
    patience=10,
    verbose=1,
    mode="max",
    restore_best_weights=True
)

In [17]:
from keras.losses import BinaryCrossentropy

In [18]:
model.compile(optimizer=opt, loss=BinaryCrossentropy(), metrics=['accuracy'])
hist = model.fit(x_train, y_train, batch_size = 100, epochs=100, callbacks=[callback])

Epoch 1/100
200/200 [==============================] - 16s 41ms/step - loss: 0.6117 - accuracy: 0.6508
Epoch 2/100
200/200 [==============================] - 8s 40ms/step - loss: 0.4588 - accuracy: 0.7827
Epoch 3/100
200/200 [==============================] - 8s 40ms/step - loss: 0.3757 - accuracy: 0.8285
Epoch 4/100
200/200 [==============================] - 8s 38ms/step - loss: 0.3086 - accuracy: 0.8640
Epoch 5/100
200/200 [==============================] - 8s 38ms/step - loss: 0.2613 - accuracy: 0.8889
Epoch 6/100
200/200 [==============================] - 8s 38ms/step - loss: 0.2187 - accuracy: 0.9067
Epoch 7/100
200/200 [==============================] - 7s 36ms/step - loss: 0.1826 - accuracy: 0.9270
Epoch 8/100
200/200 [==============================] - 7s 36ms/step - loss: 0.1422 - accuracy: 0.9437
Epoch 9/100
200/200 [==============================] - 7s 35ms/step - loss: 0.1139 - accuracy: 0.9537
Epoch 10/100
200/200 [==============================] - 7s 35ms/step - loss: 0.09

In [19]:
validation = model.evaluate(x_valid, y_valid, batch_size=100)
validation

50/50 [==============================] - 1s 18ms/step - loss: 0.5706 - accuracy: 0.9070


[0.5706122517585754, 0.9070000052452087]